<a href="https://colab.research.google.com/github/fukuokaj-st-council/FormsToSpotifyPlaylist/blob/main/Req_to_Spotipy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FormToSpotifyPlaylist v0.2

## 【情報】

### フォームが完成するまで使用不可状態

|リリース情報| |
| - | - |
|リリース日|2024/06/02|
|作成者|前田崇見|
|バージョン|0.2|
| --- | --- |
|**API情報**|**以下公開禁止**|
|Client ID|`083004bdf7a84b14a9024cb7779b762b`|
|Client Secret|`881e0c2af88e4d0c873fca6d3e30a0ef`|
|Redirect URI|`http://localhost:3000/`|

## 【現在使用不可状態】

## 【ツール本体】

In [ ]:
# @title #
# @markdown ### ←ここから実行
# @markdown ### 【使用不可】
from tqdm.notebook import tqdm as tqdm
from time import sleep
import os
tqdm.write("===================\nFormToPlaylist v0.2\n===================")
os._exit(1)
tqdm.write("パッケージ準備中...")
for i in tqdm(range(2),smoothing=0):
  if (i==0):
    !pip install -q spotipy
    import spotipy
  elif (i==1):
    !pip install -q openpyxl
    import openpyxl as xl
    import pprint
    import math
tqdm.write("Spotify利用準備中...")
client_id = "083004bdf7a84b14a9024cb7779b762b"
client_secret = "881e0c2af88e4d0c873fca6d3e30a0ef"
redirect_url = "http://localhost:3000/"
scope = "user-library-modify user-library-read"
sp_oauth = spotipy.SpotifyOAuth(client_id,client_secret,redirect_url,scope=scope)
try:
  token_info = sp_oauth.get_cached_token()
  sp = spotipy.Spotify(token_info['access_token'])
  if sp_oauth.is_token_expired(token_info) == True:
    token_info = sp_oauth.refresh_access_token(token_info['refresh_token'])
except:
    tqdm.write("トークンがありません：次の通り操作してください。")
    tqdm.write("  1.これから表示されるURLに移動してください。")
    tqdm.write("  2.しばらく待つと、「申し訳ありません。このページに到達できません」などと表示されます。")
    tqdm.write("  3.その状態のページのURLをコピーします。")
    tqdm.write("  4.次に表示されるボックスに貼り付け、Enterキーを押してください。")
    tqdm.write(sp_oauth.get_authorize_url())
    code = sp_oauth.get_authorization_code(input())
    token_info = sp_oauth.get_access_token(code)
tqdm.write("========\n準備完了\n========")
tqdm.write("リクエスト曲情報のコピー中...")
!wget "https://takaokaedu-my.sharepoint.com/:x:/g/personal/171114_takaoka-city_ed_jp/EecMEY5ls05EiXvM5mUtdn4BSb7WAb5c26xWxOn848NLcA?e=gypMfu&download=1" -q -O request_song.xlsx
tqdm.write("\b OK")
tqdm.write("リクエスト曲情報の読み込み中(この処理には時間がかかります)...")
wb = xl.load_workbook("request_song.xlsx",False,False,True,False)
tqdm.write("\b OK")
sheet = wb['@']
songlist = []
artistlist = []
tqdm.write("曲名・歌手名情報の読み込み中...")
for i in tqdm(range(1048674),smoothing=0):
  songlist.append(sheet.cell(row=i+2,column=7).value)
  if songlist[-1] == None:
    songlist.pop()
    break;
for i in tqdm(range(1048674),smoothing=0):
  artistlist.append(sheet.cell(row=i+2,column=8).value)
  if artistlist[-1] == None:
    artistlist.pop()
    break;
tqdm.write("再生済み曲の整理中...")
for i in tqdm(range(len(songlist)),smoothing=0):
  if (sheet.cell(row=i+2,column=9).value) == 1:
    songlist[i] = True
    artistlist[i] = True
while True in songlist:
  songlist.remove(True)
  artistlist.remove(True)
sp = spotipy.Spotify(client_credentials_manager=spotipy.SpotifyClientCredentials(client_id,client_secret))
idlist = []
tqdm.write("曲情報の検索中...")
for i in tqdm(range(len(songlist))):
  idlist.append(sp.search(songlist[i]+" "+artistlist[i])['tracks']['items'][0]['id'])
sp = spotipy.Spotify(auth=token_info['access_token'])
playlist_b = []
tqdm.write("プレイリストの削除中...")
for i in tqdm(range(math.ceil((sp.current_user_saved_tracks(limit=1)['total'])/50))):
  playlist_b_raw = sp.current_user_saved_tracks(limit=50,offset=i)
  for j in tqdm(range(len(playlist_b_raw['items']))):
    playlist_b.append(playlist_b_raw['items'][j]['track']['id'])
if playlist_b != []:
  sp.current_user_saved_tracks_delete(playlist_b)
tqdm.write("プレイリストへの追加中...")
sp.current_user_saved_tracks_add(idlist)
tqdm.write("========\n\n完了しました。")